In [ ]:
# === Auto-setup CBB-butanol environment (v3) ===
import os, sys
from pathlib import Path

REPO_URL = "https://github.com/BEE3600-2025/BEE-3600-Assets"
REPO_DIR = Path("/content/BEE-3600-Assets")
PROJECT_DIR = REPO_DIR / "CBB-butanol"

# Ensure /content
os.makedirs("/content", exist_ok=True)
os.chdir("/content")

# Clone or update
if not REPO_DIR.exists():
    print("Cloning repository...")
    !git clone -q {REPO_URL}
else:
    print("Repository exists; pulling latest...")
    %cd {REPO_DIR}
    !git pull -q
    %cd /content

# Git LFS (best effort)
%cd {REPO_DIR}
!git lfs version || true
!git lfs install --skip-repo || true
!git lfs pull || true

# Enter project
%cd {PROJECT_DIR}

# Put project subfolder and repo root at the FRONT of sys.path
for p in [str(PROJECT_DIR), str(REPO_DIR)]:
    if p in sys.path:
        sys.path.remove(p)
for p in reversed([str(PROJECT_DIR), str(REPO_DIR)]):
    sys.path.insert(0, p)

print("CWD:", os.getcwd())
print("PYTHONPATH head:", sys.path[:3])

# Dependencies
import pathlib
installed = False
for req in [pathlib.Path("requirements.txt"), pathlib.Path("../requirements.txt")]:
    if req.exists():
        print(f"Installing from {req} ...")
        !python -m pip -q install -r "{req}"
        installed = True
        break
if not installed:
    print("No requirements.txt found; installing a safe default stack...")
    !python -m pip -q install numpy pandas matplotlib scipy ipywidgets openpyxl biopython

print("Environment ready.")


In [ ]:
# Sanity check: ensure we're importing the correct 'utils' (from this repo)
import utils, inspect
print("utils module file:", getattr(utils, "__file__", None))

import importlib
balanceUtils = importlib.import_module("utils.balanceUtils")
print("balanceUtils file:", getattr(balanceUtils, "__file__", None))

# Show candidate function names so we can match the expected API
names = [n for n in dir(balanceUtils) if not n.startswith("_")]
print("Available in balanceUtils:", names[:50])

# If the expected function isn't present, fail fast with a helpful message
NEEDED = ["SolveFluxBalanceEquation","ConvertIndexedSMatrix","ImportIOStatus"]
missing = [n for n in NEEDED if not hasattr(balanceUtils, n)]
if missing:
    raise ImportError(f"balanceUtils is loaded from {balanceUtils.__file__}, but missing: {missing}. "
                      f"Choose from: {names[:50]} or adjust the import names to match the file.")

In [1]:
import re
import os.path
import pdb

from numpy import unique, int8, array

from utils.balanceUtils import SolveFluxBalanceEquation, ConvertIndexedSMatrix, ImportIOStatus, \
ImportReactionFile, GenerateIndexedSMatrixT, PrintStoichiometry, GenerateMergedIOStatusList, \
Get_IO_Status_for_Compound_List, ExportUniqueCompoundsWithIOStatus, GenerateIOStatusList, ParseReactionList

from utils.vectorOutput4 import generateOutputMatrixWithHeaders, writeOutputMatrix, Write_SMatrix
from utils.specutils12 import ensure_dir

# Reaction List

In [2]:
reactionList = \
['CO2 + Ribulose15BP + H2O -> 2*3PG', \
 '3PG + ATP -> 13BPG + ADP', \
 '13BPG + NADH + Hplus -> G3P + P + NADplus', \
 'G3P + GlyceronePhosph -> F16BP', \
 'F16BP + H2O -> F6P + P', \
 'F6P + G3P -> E4P + X5P', \
 'G3P -> GlyceronePhosph', \
 'E4P + GlyceronePhosph -> SH17BP', \
 'X5P -> Ribulose5P', \
 'SH17BP + H2O -> SH7P + P', \
 'SH7P + G3P -> Ribose5P + X5P', \
 'Ribose5P -> Ribulose5P', \
 'ATP + Ribulose5P -> ADP + Ribulose15BP', \
 '3PG -> 2PG', \
 '2PG -> PEP + H2O', \
 'ADP + PEP + P -> Pyruvate + ATP', \
 'NADplus + Pyruvate + CoA -> NADH + Acetyl-CoA + CO2 + 2*Hplus', \
 '2*Acetyl-CoA -> CoA + Acetoacetyl-CoA', \
 'Acetoacetyl-CoA + NADH + Hplus -> Hydroxybutanoyl-CoA + NADplus', \
 'Hydroxybutanoyl-CoA -> Crotonoyl-CoA + H2O', \
 'Crotonoyl-CoA + NADH + Hplus -> Butanoyl-CoA + NADplus', \
 'Butanoyl-CoA + NADH + Hplus -> Butanal + CoA + NADplus', \
 'Butanal + NADH + Hplus -> 1-Butanol + NADplus' \
]

In [3]:
len(reactionList)

23

In [4]:
reactionList[-8]

'ADP + PEP + P -> Pyruvate + ATP'

In [5]:
reactantsToGet = ['ATP', 'NADH', 'CO2'] 

# Prepare Stoichiometric Matrix

Figure out the unique compounds, reactions, and calculate a stoichiomietric matrix. 

In [6]:
uniqueCompounds, reactions, sMatrixTKeyIndexed = \
ParseReactionList(reactionList, reactionArrow='->')

In [7]:
uniqueCompoundsIOStatusProposed = GenerateIOStatusList(uniqueCompounds)
uniqueCompoundsIOStatusProposed

[['1-Butanol', 'Intermediate'],
 ['13BPG', 'Intermediate'],
 ['2PG', 'Intermediate'],
 ['3PG', 'Intermediate'],
 ['ADP', 'Intermediate'],
 ['ATP', 'Input'],
 ['Acetoacetyl-CoA', 'Intermediate'],
 ['Acetyl-CoA', 'Intermediate'],
 ['Butanal', 'Intermediate'],
 ['Butanoyl-CoA', 'Intermediate'],
 ['CO2', 'Input/Output'],
 ['CoA', 'Intermediate'],
 ['Crotonoyl-CoA', 'Intermediate'],
 ['E4P', 'Intermediate'],
 ['F16BP', 'Intermediate'],
 ['F6P', 'Intermediate'],
 ['G3P', 'Intermediate'],
 ['GlyceronePhosph', 'Intermediate'],
 ['H2O', 'Input/Output'],
 ['Hplus', 'Intermediate'],
 ['Hydroxybutanoyl-CoA', 'Intermediate'],
 ['NADH', 'Input'],
 ['NADplus', 'Intermediate'],
 ['P', 'Intermediate'],
 ['PEP', 'Intermediate'],
 ['Pyruvate', 'Intermediate'],
 ['Ribose5P', 'Intermediate'],
 ['Ribulose15BP', 'Intermediate'],
 ['Ribulose5P', 'Intermediate'],
 ['SH17BP', 'Intermediate'],
 ['SH7P', 'Intermediate'],
 ['X5P', 'Intermediate']]

# Edit This

Make your best guess about the input/output status of each compound. Input/Output, Input, and Output are all the same from the point of view of the code. The crucial difference is between Target (one allowed); Intermediate; and Input|Output. 

In [8]:
uniqueCompoundsIOStatusEdit = \
[['1-Butanol', 'Target'],
 ['13BPG', 'Intermediate'],
 ['2PG', 'Intermediate'],
 ['3PG', 'Intermediate'],
 ['ADP', 'Output'],
 ['ATP', 'Input'],
 ['Acetoacetyl-CoA', 'Intermediate'],
 ['Acetyl-CoA', 'Intermediate'],
 ['Butanal', 'Intermediate'],
 ['Butanoyl-CoA', 'Intermediate'],
 ['CO2', 'Input/Output'],
 ['CoA', 'Intermediate'],
 ['Crotonoyl-CoA', 'Intermediate'],
 ['E4P', 'Intermediate'],
 ['F16BP', 'Intermediate'],
 ['F6P', 'Intermediate'],
 ['G3P', 'Intermediate'],
 ['GlyceronePhosph', 'Intermediate'],
 ['H2O', 'Input/Output'],
 ['Hplus', 'Output'],
 ['Hydroxybutanoyl-CoA', 'Intermediate'],
 ['NADH', 'Input'],
 ['NADplus', 'Output'],
 ['P', 'Output'],
 ['PEP', 'Intermediate'],
 ['Pyruvate', 'Intermediate'],
 ['Ribose5P', 'Intermediate'],
 ['Ribulose15BP', 'Intermediate'],
 ['Ribulose5P', 'Intermediate'],
 ['SH17BP', 'Intermediate'],
 ['SH7P', 'Intermediate'],
 ['X5P', 'Intermediate']]

i = 0
ioStatusEdit = []
while i < len(uniqueCompoundsIOStatusEdit):
    ioStatusEdit.append(uniqueCompoundsIOStatusEdit[i][1])
    i += 1

# Balance the Stoichiometric Matrix

In [9]:
sMatrixT = ConvertIndexedSMatrix(sMatrixTKeyIndexed, uniqueCompounds)
sMatrix = sMatrixT.transpose()


[fVectorOpt, cDotVectorOpt, cDotVectorOptNorm, result] = \
SolveFluxBalanceEquation(sMatrix, reactions, uniqueCompounds, ioStatusEdit)

In [10]:
[fVectorOpt, cDotVectorOpt, cDotVectorOptNorm, result];

In [11]:
i = 0
printEverything = True
while i < len(uniqueCompounds):
        ioStatus = ioStatusEdit[i]
        compound = uniqueCompounds[i]
        if printEverything == False and (compound in reactantsToGet) or ioStatus == 'Target':
            print(uniqueCompounds[i] + ':\t' + ioStatus + '\t' + "%.1f" % cDotVectorOptNorm[i])
        elif printEverything == True:
            print(uniqueCompounds[i] + ':\t' + ioStatus + '\t' + "%.1f" % cDotVectorOptNorm[i])
        i += 1

1-Butanol:	Target	1.0
13BPG:	Intermediate	-0.0
2PG:	Intermediate	-0.0
3PG:	Intermediate	-0.0
ADP:	Output	14.0
ATP:	Input	-14.0
Acetoacetyl-CoA:	Intermediate	0.0
Acetyl-CoA:	Intermediate	0.0
Butanal:	Intermediate	-0.0
Butanoyl-CoA:	Intermediate	-0.0
CO2:	Input/Output	-4.0
CoA:	Intermediate	-0.0
Crotonoyl-CoA:	Intermediate	-0.0
E4P:	Intermediate	-0.0
F16BP:	Intermediate	0.0
F6P:	Intermediate	-0.0
G3P:	Intermediate	-0.0
GlyceronePhosph:	Intermediate	0.0
H2O:	Input/Output	-7.0
Hplus:	Output	-10.0
Hydroxybutanoyl-CoA:	Intermediate	0.0
NADH:	Input	-12.0
NADplus:	Output	12.0
P:	Output	12.0
PEP:	Intermediate	-0.0
Pyruvate:	Intermediate	-0.0
Ribose5P:	Intermediate	0.0
Ribulose15BP:	Intermediate	-0.0
Ribulose5P:	Intermediate	0.0
SH17BP:	Intermediate	0.0
SH7P:	Intermediate	0.0
X5P:	Intermediate	0.0


In [12]:
fVectorOpt / fVectorOpt[-2]

array([ 6.00145774, 10.00399835, 10.00433027,  2.0004976 ,  2.0004781 ,
        2.00069612,  4.00259605,  2.00158414,  4.00108372,  2.00138923,
        2.00072924,  2.00040462,  6.00126612,  1.99912786,  1.9995726 ,
        2.00025092,  2.00040743,  1.00008536,  0.99904728,  0.99874093,
        0.99913728,  1.        ,  1.00031172])

In [13]:
sMatrix;

In [14]:
reactionList[-1]

'Butanal + NADH + Hplus -> 1-Butanol + NADplus'